# FINAL MODEL

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score

Import the datasets

In [2]:
train = pd.read_csv('training_processed.csv')
X_train = train.drop(['ACTIVE'], axis=1) 
y_train = train['ACTIVE']

val = pd.read_csv('validation_processed.csv')
X_val = val.drop(['ACTIVE'], axis=1) 
y_val = val['ACTIVE']

total = pd.read_csv('total_processed.csv')
X_total = total.drop(['ACTIVE'], axis=1) 
y_total = total['ACTIVE']

X_test = pd.read_csv('testing_processed_1.csv') ## TOTAL PROCESSED WITH PCA applied only on training data, then applied on validation data and test data
X_test_2 = pd.read_csv('testing_processed_2.csv') ## TOTAL PROCESSED WITH PCA applied on training and validation data, then applied on test data

Double-check the parameter settings on the validation set

In [3]:
ann = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, alpha=0.0001, activation='relu',solver='adam', random_state=1, learning_rate='constant')
ann.fit(X_train, y_train)
ann_labels = ann.predict(X_val)# predicted labels
ann_score = ann.score(X_val, y_val)
ann_pred = ann.predict_proba(X_val)

ann_auc = roc_auc_score(y_val, ann_pred[:,1])

# print the results
print('ANN Accuracy: ', ann_score)
print('ANN AUC: ', ann_auc)

ANN Accuracy:  0.9887879175732753
ANN AUC:  0.7633064292573168


Train the ANN on the full dataset and predict the test labels

In [29]:
ann = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, alpha=0.0001, activation='relu',solver='adam', random_state=1, learning_rate='constant')
ann.fit(X_total, y_total)
ann_labels = ann.predict(X_test)# predicted labels
ann_pred = ann.predict_proba(X_test)
ann_pred = ann_pred[:,1] # take only the active class

print(ann_labels)
print(ann_pred)


[0. 0. 0. ... 0. 0. 0.]
[0.00498241 0.01477038 0.00574764 ... 0.03734055 0.00514051 0.00293118]


In [30]:
ann = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, alpha=0.0001, activation='relu',solver='adam', random_state=1, learning_rate='constant')
ann.fit(X_total, y_total)
ann_labels_2 = ann.predict(X_test_2)# predicted labels
ann_pred_2 = ann.predict_proba(X_test_2)
ann_pred_2 = ann_pred_2[:,1] # take only the active class


print(ann_labels_2)
print(ann_pred_2)

[0. 0. 0. ... 0. 0. 0.]
[0.00708067 0.00205526 0.00178856 ... 0.01157009 0.05702776 0.01006459]


In [31]:
# add first row to array
ann_pred = np.insert(ann_pred, 0, 0.77) # 0.77 is the estimated auc
ann_pred_2 = np.insert(ann_pred_2, 0, 0.77) # 0.77 is the estimated auc
print(ann_pred)

np.savetxt('6.txt', ann_pred)
np.savetxt('6(with pca fitted to training and validation).txt', ann_pred_2)

[0.77       0.00498241 0.01477038 ... 0.03734055 0.00514051 0.00293118]


In [32]:
predictions_df = pd.read_csv("6.txt", header=None)
print(predictions_df.shape == (52087, 1))
print(np.all((predictions_df.values >= 0) & (predictions_df.values <= 1)))

True
True


In [33]:
predictions_df = pd.read_csv("6(with pca fitted to training and validation).txt", header=None)
print(predictions_df.shape == (52087, 1))
print(np.all((predictions_df.values >= 0) & (predictions_df.values <= 1)))

True
True
